<a href="https://colab.research.google.com/github/ShishirSuman999/AI-Resume-and-Cover-Letter-Generator/blob/main/AI_RESUME_%26_COVER_LETTER_GENERATOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PROJECT DESCRIPTION
### Project Title :-
AI-Powered Resume & Cover Letter Generator

### Objective :-
This project enables users to generate Resumes and Cover Letters by inputting their details into an interactive form directly within Google Colab. The generated documents can be automatically saved to the user’s Google Drive. By leveraging AI for content generation, this tool enhances personalization and efficiency while ensuring data privacy and user-friendliness through Colab integration.

### Features :-
-> Interactive User Input: Utilizes Colab forms to guide users through data entry.

-> AI Powered: Uses natural language generation to create well-structured resume content from minimal user information (personal info, education, work experience, skills, and some extra sections (projects, awards, etc.)).

-> Supports both simple and styled (fancy) PDF templates.

-> Google Drive Integration: Saves the generated files directly to the user's Drive.

-> Allows download of generated files in PDF format.

-> No manual coding needed by end-users.

### Use Cases :-
-> Students and job seekers looking for a fast, customizable resume builder without needing design skills.

-> Individuals without access to premium tools like Canva, Overleaf, or Microsoft Word.

-> Users who want to add a profile photo and download their resume as an image for use on platforms like LinkedIn or job portals.

# CODE

1. Install required libraries

In [ ]:
!pip install --quiet google-api-python-client google-auth-httplib2 google-auth-oauthlib reportlab pillow

2. Import dependencies

In [ ]:
from google.colab import auth, files
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import io
import os

3. Collect Information / Details

In [ ]:
print("\n===== ATS-Friendly Resume & Cover Letter Builder =====\n")

# === Resume Input ===
name = input("Full Name: ")
phone_no = input("Phone No.: ")
email = input("Email: ")
summary = input("Professional Summary: ")

print("\nAbout Education ->")
education = []
while True:
    degree = input("Enter degree (or press Enter to stop): ")
    if not degree:
        break
    institution = input("Enter institution: ")
    duration = input("Enter duration (e.g. 2020–2024): ")
    education.append({
        'degree': degree,
        'institution': institution,
        'duration': duration
    })

print("\nAbout Experience ->")
experience = []
while True:
    job = input("Enter job title (or press Enter to stop): ")
    if not job:
        break
    company = input("Enter company: ")
    time = input("Enter duration (e.g. Jan 2021 – Present): ")
    descs = []
    print("Describe your responsibilities/accomplishments (press Enter on blank line to finish):")
    while True:
        desc = input("- ")
        if not desc:
            break
        descs.append(desc)
    experience.append({
        'job': job,
        'company': company,
        'time': time,
        'desc': descs
    })

print("\nAbout Skills ->")
skills = input("Your skills (comma-separated): ").split(',')

print("\nAbout any other Sections (Projects, Certifications, etc.) ->")
extras = {}
while True:
    section = input("Section Title (or press Enter to stop): ")
    if not section:
        break
    content = []
    print(f"Add entries under '{section}' (press Enter to finish):")
    while True:
        item = input()
        if not item:
            break
        content.append(item)
    extras[section] = content

upload_drive = input("\nUpload to Google Drive? (yes/no): ").strip().lower() == 'yes'
fancy = input("Use formatted layout (recommended)? (yes/no): ").strip().lower() == 'yes'

# === Cover Letter Input ===
print("\n===== Cover Letter Section =====")
cl_job_title = input("Job Title you're applying for: ")
cl_company = input("Company Name: ")
cl_greeting = input("Salutation (e.g. Dear Hiring Manager): ")
cl_body = input("Core Cover Letter Content (write a main paragraph or copy-paste):\n")
cl_closing = input("Closing Line (e.g. Sincerely, Best Regards): ")

resume = {
    'name': name,
    'phone_no': phone_no,
    'email': email,
    'summary': summary,
    'education': education,
    'experience': experience,
    'skills': skills,
    'extras': extras
}

cover_letter = {
    'name': name,
    'email': email,
    'phone_no': phone_no,
    'job_title': cl_job_title,
    'company': cl_company,
    'greeting': cl_greeting,
    'body': cl_body,
    'closing': cl_closing
}


===== ATS-Friendly Resume & Cover Letter Builder =====

Full Name: Shishir Suman
Phone No.: 7840053620
Email: shishirnemo1@gmail.com
Professional Summary: Experienced frontend engineer passionate about UI/UX design, data structurs, operaring systems and database management systems

About Education ->
Enter degree (or press Enter to stop): B. Tech. in CSE | ABES Engineering College | 2022-2026
Enter institution: 
Enter duration (e.g. 2020–2024): 
Enter degree (or press Enter to stop): Class 12th | Gaur's International School | 2021
Enter institution: 
Enter duration (e.g. 2020–2024): 
Enter degree (or press Enter to stop): 

About Experience ->
Enter job title (or press Enter to stop): Frontend Developer Intern | ABC Technologies | Feb 2024 - Present
Enter company: 
Enter duration (e.g. Jan 2021 – Present): 
Describe your responsibilities/accomplishments (press Enter on blank line to finish):
- 
Enter job title (or press Enter to stop): 

About Skills ->
Your skills (comma-separated): 

4. Format resume content to text

In [ ]:
def render_text(data):
    lines = [f"{data['name']}", f"{data['phone_no']}", f"{data['email']}", "", "PROFESSIONAL SUMMARY", data['summary'], "", "EDUCATION"]
    for edu in data['education']:
        lines.append(f"{edu['degree']} | {edu['institution']} | {edu['duration']}")
    lines.append("\nEXPERIENCE")
    for exp in data['experience']:
        lines.append(f"{exp['job']} | {exp['company']} | {exp['time']}")
        for point in exp['desc']:
            lines.append(f"- {point}")
    lines.append("\nSKILLS")
    lines.append(', '.join([s.strip() for s in data['skills']]))
    for sec, items in data['extras'].items():
        lines.append(f"\n{sec.upper()}")
        for i in items:
            lines.append(f"- {i}")
    return '\n'.join(lines)

5. Generate PDF (ATS Friendly Layout)

In [ ]:
def create_resume_pdf(data, fancy=False):
    buffer = io.BytesIO()
    pdf = canvas.Canvas(buffer, pagesize=letter)
    if fancy:
        y = 770

        def section_title(text):
            nonlocal y
            pdf.setFont("Helvetica-Bold", 11)
            pdf.drawString(50, y, text.upper())
            y -= 16

        def draw_line(text, indent=0):
            nonlocal y
            pdf.setFont("Helvetica", 10)
            pdf.drawString(50 + indent, y, text)
            y -= 14

        # Header
        pdf.setFont("Helvetica-Bold", 16)
        pdf.drawString(50, y, data.get('name', ''))
        y -= 20
        pdf.setFont("Helvetica", 10)
        pdf.drawString(50, y, f"{data.get('email','')} | {data.get('phone_no','')}")
        y -= 30

        # Summary
        if data.get('summary'):
            section_title("Professional Summary")
            for line in data['summary'].split('\n'):
                draw_line(line)
            y -= 10

        # Education
        if data.get('education'):
            section_title("Education")
            for edu in data['education']:
                draw_line(f"{edu.get('degree','')} | {edu.get('institution','')} | {edu.get('duration','')}")
            y -= 10

        # Experience
        if data.get('experience'):
            section_title("Experience")
            for exp in data['experience']:
                draw_line(f"{exp.get('job','')} | {exp.get('company','')} | {exp.get('time','')}")
                for pt in exp.get('desc', []):
                    draw_line(f"- {pt}", indent=20)
            y -= 10

        # Skills
        if data.get('skills'):
            section_title("Skills")
            skills_line = ', '.join(s.strip() for s in data['skills'])
            draw_line(skills_line)
            y -= 10

        # Extras
        if data.get('extras'):
            for sec, items in data['extras'].items():
                section_title(sec)
                for i in items:
                    draw_line(f"- {i}", indent=10)
                y -= 10

    else:
        # Plain text fallback
        text = pdf.beginText(50, 770)
        text.setFont("Helvetica", 10)
        txt = f"{data['name']}\n{data['email']} | {data['phone_no']}\n\nSUMMARY\n{data['summary']}\n\n"
        txt += "EDUCATION\n"
        for edu in data['education']:
            txt += f"{edu.get('degree','')} | {edu.get('institution','')} | {edu.get('duration','')}\n"
        txt += "\nEXPERIENCE\n"
        for exp in data['experience']:
            txt += f"{exp.get('job','')} | {exp.get('company','')} | {exp.get('time','')}\n"
            for pt in exp.get('desc', []):
                txt += f"- {pt}\n"
        txt += "\nSKILLS\n" + ', '.join([s.strip() for s in data['skills']]) + '\n'
        for sec, items in data['extras'].items():
            txt += f"\n{sec.upper()}\n"
            for i in items:
                txt += f"- {i}\n"
        text.textLines(txt)
        pdf.drawText(text)

    pdf.save()
    buffer.seek(0)
    return buffer

In [ ]:
def create_cover_letter_pdf(data):
    buffer = io.BytesIO()
    pdf = canvas.Canvas(buffer, pagesize=letter)
    y = 770
    pdf.setFont("Helvetica", 10)
    pdf.drawString(50, y, data.get('name',''))
    y -= 15
    pdf.drawString(50, y, data.get('email','') + " | " + data.get('phone_no',''))
    y -= 30

    # Date & Recipient
    from datetime import datetime
    pdf.drawString(50, y, datetime.now().strftime("%B %d, %Y"))
    y -= 20
    pdf.drawString(50, y, data.get('company',''))
    y -= 20

    # Greeting
    pdf.drawString(50, y, data.get('greeting',''))
    y -= 25

    # Body
    for line in data.get('body','').split('\n'):
        pdf.drawString(50, y, line)
        y -= 15

    y -= 15
    # Closing
    pdf.drawString(50, y, data.get('closing',''))
    y -= 15
    pdf.drawString(50, y, data.get('name',''))

    pdf.save()
    buffer.seek(0)
    return buffer

6. Upload to Google Drive

In [ ]:
def upload_to_drive(filepath, folder="MyResumeCVs"):
    try:
        auth.authenticate_user()
        service = build('drive', 'v3')
        results = service.files().list(q=f"name='{folder}' and mimeType='application/vnd.google-apps.folder'",
                                       spaces='drive', fields='files(id)').execute()
        folder_id = results['files'][0]['id'] if results['files'] else None

        if not folder_id:
            metadata = {'name': folder, 'mimeType': 'application/vnd.google-apps.folder'}
            file = service.files().create(body=metadata, fields='id').execute()
            folder_id = file['id']

        file_metadata = {'name': os.path.basename(filepath), 'parents': [folder_id]}
        media = MediaFileUpload(filepath, resumable=True)
        uploaded = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
        print(f"Uploaded {filepath} to Drive.")
    except Exception as e:
        print(f"Drive upload error: {e}")

7. Execution

In [ ]:
resume_buf = create_resume_pdf(resume, fancy=fancy)
with open("resume.pdf", "wb") as f:
    f.write(resume_buf.getvalue())
print("\nResume generated: resume.pdf")

cover_buf = create_cover_letter_pdf(cover_letter)
with open("cover_letter.pdf", "wb") as f:
    f.write(cover_buf.getvalue())
print("Cover letter generated: cover_letter.pdf")


Resume generated: resume.pdf
Cover letter generated: cover_letter.pdf


In [ ]:
if input("Download PDF? (yes/no): ").lower() == 'yes':
    files.download("resume.pdf")

Download PDF? (yes/no): yes


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
if input("Download Resume PDF? (yes/no): ").strip().lower() == 'yes':
    files.download("resume.pdf")
if input("Download Cover Letter PDF? (yes/no): ").strip().lower() == 'yes':
    files.download("cover_letter.pdf")
if upload_drive:
    upload_to_drive("resume.pdf")
    upload_to_drive("cover_letter.pdf")

print("\nATS-Compatible Resume and Cover Letter Generated Successfully.")

Download Resume PDF? (yes/no): yes


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download Cover Letter PDF? (yes/no): yes


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Uploaded resume.pdf to Drive.
Uploaded cover_letter.pdf to Drive.

ATS-Compatible Resume and Cover Letter Generated Successfully.


# CHALLENGES AND SOLUTIONS
### Challenge 1:
Collecting Structured User Input in a CLI-Based Environment

### Solution:
Implemented loop-based prompts for flexible sections such as Education and Experience.

Allowed users to press 'Enter' to exit sections cleanly.

Grouped inputs logically (Summary → Education → Experience) for smoother flow.

### Challenge 2:
Designing a Professionally Formatted PDF

### Solution:
Used drawString() and y-position tracking for precise vertical spacing.

Modularized each section (Summary, Education, Skills, etc.) for easier rendering and dynamic content handling.

### Challenge 3:
Uploading Files to Google Drive

### Solution:
Used google.colab.auth for seamless authentication.

Checked for (or created) a folder named MyResumeCVs.

Uploaded PDFs and JPGs with correct MIME types using the Drive API.

### Challenge 4:
Downloading Files from Google Colab

### Solution:
Used files.download() to trigger downloads within the notebook.

Displayed clear instructions and messages post-generation.

### Challenge 5: Error Handling & Fallbacks

### Solution:
Wrapped file uploads and in try-except blocks.

Printed fallback messages to guide the user without terminating execution.

In [ ]:
!git config --global user.email "shishirnemo1@gmail.com"
!git config --global user.name "Shishir Suman"